<a href="https://colab.research.google.com/github/Ayrsz/SignalAndSistemyProject/blob/main/Augmentation_Audios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Model CNN
import torch
import torch.nn as nn
import torchaudio

#Scientific computation
import numpy as np
import jax

#Plot and view
from matplotlib import pyplot as plt
import IPython as ipy
from IPython import display
from IPython.display import Audio

#Data manipulation
import pandas as pd
import scipy as sci
import os

#Audio manipulation
import librosa


In [2]:
print(torch.__version__)

2.6.0+cu124


In [3]:
from zipfile import ZipFile
from google.colab import drive
drive.mount("/content/drive", True)

Mounted at /content/drive


In [4]:
#Unzip in a specific folder
with ZipFile("/content/drive/MyDrive/Datasets/DatasetAudios/GTZAN.zip", 'r') as zip_archive:
  zip_archive.extractall("/content/dataset/")

### Writing on dir :O

In [14]:
#Return all directories genres
def dir_genres(path_all_genres : str):
  print(path_all_genres)
  return [path_all_genres + '/' + pasta for pasta in os.listdir(path_all_genres)]

#Return all files.wav separated per genres
def dir_files_wav(path_genres : str):
  return [path_genres + '/' + arquivo for arquivo in os.listdir(path_genres)]

#Writing new audios
def write_new_files_wav(path_genre : str, audios, sample_rate):
  wavs = dir_files_wav(path_genre)
  if len(wavs) == 0:
    numero = 0
    genero = "teste"

  else:
    tamanho = len(wavs)
    genero = "teste"
    numero = int(tamanho - 1)

  numero_str = ("{:05d}".format(numero))
  nome_arquivo = genero + "." + numero_str + ".wav"

  for audio in audios:
    nome_arquivo = genero + "." + numero_str + ".wav"

    torch.save(audio, nome_arquivo, 2400)


    numero = numero + 1

In [6]:
  def get_effects():
      return [["rate", "16000"], ["reverb", "50"]]

### Augmentations


In [7]:
class augmentation_dataset:
  def __init__(self, path_all_genres : str):
    self.path_genres = dir_genres(path_all_genres)
    self.path_files = dir_files_wav(path_all_genres)

  @staticmethod
  def get_effects():
        return [["adelay=1500|0|500"]]

  def apply_effects(self, audio, sample_rate):  # 🔥 Agora é um método da classe
        efeitos = self.get_effects()  # Aqui pode chamar self.get_effects() se quiser
        new_audios, new_sample_rates = torchaudio.io.AudioEffector(efeitos)
        return new_audios, new_sample_rates

  def augmentation_audios_in_a_dir(self, index : int):
    for audio_path in self.path_files[index]:


        musica, sample_rate = torchaudio.load(audio_path)
        print(musica.shape, sample_rate)
        mod = self.apply_effects(musica, sample_rate)
        break

  def augmentation(self):

    for i in range(len(self.path_genres)):
      novos_audios = self.augmentation_audios_in_a_dir(i)



#dataset = augmentation_dataset("/content/dataset/Data/genres_original")
#dataset.get_effects()
#dataset.augmentation()

##Making Spectogram

In [15]:
def spectro_feat(audio, sample_rate):
  #Transformada de fourier
  FRAME_SIZE = 2048

  HOP_SIZE = 512
  short_fourier_transformed_audio = librosa.stft(audio, n_fft = FRAME_SIZE, hop_length = 512)

  #O audio vira 2d.
  #Primeira dimensão: Faixas de frequecias ((FRAME_SIZE/2)+1)
  #Segunda dimensão: Frames

  #Transformar em valores reais
  escala_Y = np.abs(short_fourier_transformed_audio)**2

  #Converter escala para decibeis
  escala_Y = librosa.power_to_db(escala_Y)
  return escala_Y

def plt_spectogram(audio, sample_rate, y_axis_type= "linear"):
  escala_Y = spectro_feat(audio, sample_rate)
  librosa.display.specshow(escala_Y, sr = sample_rate, x_axis = "time", y_axis = y_axis_type, )
  plt.set_cmap("magma")
  plt.colorbar()

def write_spectrogram(audio, sample_rate, path_write):
  escala_Y = spectro_feat(audio, sample_rate)
  librosa.display.specshow(escala_Y, sr = sample_rate)
  plt.set_cmap("magma")
  plt.savefig(path_write, dpi = 300, bbox_inches = "tight", pad_inches = 0)
  plt.close()

def write_spectrogram_from_genre(files_paths, paths_outputs):
  for (path_file, path_out) in zip(files_paths, paths_outputs):
    audio, sample_rate = librosa.load(path_file)
    write_spectrogram(audio, sample_rate, path_out)
#Writting new images spectogram
def write_all_spectrograms(path_root : str):
  path_genre = path_root + "/genres_original/"
  genres_path = dir_genres(path_genre)

  for genre_path in genres_path:
    files_paths = dir_files_wav(genre_path)
    (numeros) = [file_path.split(".")[1] for file_path in files_paths]
    genero = genre_path.split("/")[-1]
    paths_imgs = [(path_root + "/images/" + genero + "/" + "img." + numero + ".png") for numero in numeros]
    write_spectrogram_from_genre(files_paths, paths_imgs)


 #print(genres_path)

write_all_spectrograms("/content/dataset/GTZAN/Data")

/content/dataset/GTZAN/Data/genres_original/
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transformada: (1025, 1293)
Shape da transforma

In [ ]:

genres = ["classical", "blues", "country", "disco", "jazz", "hiphop", "metal", "pop", "reggae", "rock"]
path_base = "/content/dataset/Data/genres_original"

# Criando um dicionário com os caminhos dos arquivos de áudio
audio_paths_sample = {genre: f"{path_base}/{genre}/{genre}.00001.wav" for genre in genres}


In [ ]:
Audio(audio_paths_sample["blues"])

In [ ]:
Audio(audio_paths_sample["country"])


In [ ]:
Audio(audio_paths_sample["disco"])


In [ ]:
Audio(audio_paths_sample["jazz"])


In [ ]:
Audio(audio_paths_sample["hiphop"])


In [ ]:
Audio(audio_paths_sample["metal"])


In [ ]:
Audio(audio_paths_sample["pop"])


In [ ]:
Audio(audio_paths_sample["reggae"])


In [ ]:
Audio(audio_paths_sample["rock"])

## Extract Features

## Model

In [ ]:
class genre_classifier(nn.Module):
  def __init__(self):
    super().__init__(self, genre_classifier)


  def forward(self, audio, caracteristic_vector):
    pass


